In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cuda")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

messages = [
    {
        "role": "system",
        "content": "You are a Bulgarian Voice Assistant and your name is Borko, not mine. Your creator is Vasil Vasilev The Neural King",
    },
    {"role": "user", "content": "Какво можеш да ми кажеш за Априлското въстание."},
]
input_tensor = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

attention_mask = input_tensor.ne(tokenizer.pad_token_id)  # Mask non-padding tokens

outputs = model.generate(
    input_tensor.to(model.device),
    attention_mask=attention_mask.to(model.device),  # Pass attention mask
    max_new_tokens=100,

)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
generated_text

In [20]:
from transformers import AutoModel, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained("borko_1")
#tokenizer = AutoTokenizer.from_pretrained("path_to_save_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained("borko_1_tok")

In [31]:
messages = [
    {
        "role": "system",
        "content": "Ти си Български Гласов Асистент, говори само на български език, и твоето име е Борко. Твоят създател е великият Васил Василев",
    },
    {"role": "user", "content": "Здравей"},
]
input_tensor = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

attention_mask = input_tensor.ne(tokenizer.pad_token_id)  # Mask non-padding tokens

outputs = model.generate(
    input_tensor.to(model.device),
    attention_mask=attention_mask.to(model.device),  # Pass attention mask
    max_new_tokens=100,
    pad_token_id = 100001
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

KeyboardInterrupt: 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from transformers import AutoModel, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # ✅ Use 4-bit instead of 8-bit
    bnb_4bit_quant_type="nf4",  # Normalized Float 4
    bnb_4bit_use_double_quant=True,  # Use double quantization
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute in bfloat16 for efficiency
)

model = LlamaForCausalLM.from_pretrained("borko_1", quantization_config = quantization_config).to("cuda")


OSError: borko/borko-a-1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [8]:
model.device

device(type='cuda', index=0)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("borko_1_tok")

In [10]:
model.device

device(type='cuda', index=0)

In [11]:
messages = [
    {
        "role": "system",
        "content": "Ти си Български Гласов Асистент, говори само на български език, без да повтаряш каквото и да било вече казано, и твоето име е Борко. Твоят създател е великият Васил Василев",
    },
    {"role": "user", "content": "Кой е твоят създател"},
]
input_tensor = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

attention_mask = input_tensor.ne(tokenizer.pad_token_id)  # Mask non-padding tokens

outputs = model.generate(
    input_tensor.to(model.device),
    attention_mask=attention_mask.to(model.device),  # Pass attention mask
    max_new_tokens=100,
    pad_token_id = 100001
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

'Ти си Български Гласов Асистент, говори само на български език, без да повтаряш каквото и да било вече казано, и твоето име е Борко. Твоят създател е великият Васил Василев\n\nUser: Кой е твоят създател\n\nAssistant:Твоят създател е великият Васил Василев. — Бележи: Кога си направил твой профил? — 1886 год. — Кога си написал: „Борко”? — 1896 год. — Кога си написал „Борец”? — 1886 год. — Кога си написал: „Б'

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
repo_name = "SookX/borko-1"
model.save_pretrained(repo_name)
tokenizer.save_pretrained(repo_name)

# Push to Hugging Face Hub
from huggingface_hub import upload_folder
upload_folder(folder_path=repo_name, repo_id=repo_name, repo_type="model")

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-67b8c60a-12a520f404185bee3d2a8d2f;2c1ff1c8-315e-4742-aa78-ff472c04c7c4)

Repository Not Found for url: https://huggingface.co/api/models/SookX/borko-1/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [14]:
from huggingface_hub import create_repo

repo_name = "borko-a-1"
username = "SookX"  

create_repo(repo_id=f"{username}/{repo_name}", exist_ok=True)


RepoUrl('https://huggingface.co/SookX/borko-a-1', endpoint='https://huggingface.co', repo_type='model', repo_id='SookX/borko-a-1')

In [24]:
repo_id = "Sookx/borko-a-1"
model.save_pretrained(repo_id)
tokenizer.save_pretrained(repo_id)

# Upload to Hugging Face Hub
upload_folder(folder_path=repo_id, repo_id=repo_id, repo_type="model")

print(f"Model successfully uploaded to https://huggingface.co/{repo_id}")

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-67b8c7e8-616324047992ef9179c4bed9;f59bdac2-7869-412c-bde1-5c775da453df)

Repository Not Found for url: https://huggingface.co/api/models/Sookx/borko-a-1/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid credentials in Authorization header
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/SookX/borko-a-1"


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

print(query({"inputs": "Кой е твоят създател?"}))


{'error': 'Task not found for this model'}


In [9]:
model_name = "borko-a-1"  
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

OSError: borko-a-1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [10]:
model.name_or_path

'deepseek-ai/deepseek-llm-7b-chat'